##### Copyright 2019 The TensorFlow Authors.

In [2]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# TensorFlow 2 quickstart for beginners

<table class="tfo-notebook-buttons" align="left">
  <td>
    <a target="_blank" href="https://www.tensorflow.org/tutorials/quickstart/beginner"><img src="https://www.tensorflow.org/images/tf_logo_32px.png" />View on TensorFlow.org</a>
  </td>
  <td>
    <a target="_blank" href="https://colab.research.google.com/github/tensorflow/docs/blob/master/site/en/tutorials/quickstart/beginner.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png" />Run in Google Colab</a>
  </td>
  <td>
    <a target="_blank" href="https://github.com/tensorflow/docs/blob/master/site/en/tutorials/quickstart/beginner.ipynb"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png" />View source on GitHub</a>
  </td>
  <td>
    <a href="https://storage.googleapis.com/tensorflow_docs/docs/site/en/tutorials/quickstart/beginner.ipynb"><img src="https://www.tensorflow.org/images/download_logo_32px.png" />Download notebook</a>
  </td>
</table>

This short introduction uses [Keras](https://www.tensorflow.org/guide/keras/overview) to:

1. Load a prebuilt dataset.
1. Build a neural network machine learning model that classifies images.
2. Train this neural network.
3. Evaluate the accuracy of the model.

This tutorial is a [Google Colaboratory](https://colab.research.google.com/notebooks/welcome.ipynb) notebook. Python programs are run directly in the browser—a great way to learn and use TensorFlow. To follow this tutorial, run the notebook in Google Colab by clicking the button at the top of this page.

1. In Colab, connect to a Python runtime: At the top-right of the menu bar, select *CONNECT*.
2. To run all the code in the notebook, select **Runtime** > **Run all**. To run the code cells one at a time, hover over each cell and select the **Run cell** icon.

![Run cell icon](https://github.com/tensorflow/docs/blob/master/site/en/tutorials/quickstart/images/beginner/run_cell_icon.png?raw=1)

## Set up TensorFlow

Import TensorFlow into your program to get started:

In [3]:
import tensorflow as tf
print("TensorFlow version:", tf.__version__)

TensorFlow version: 2.18.0


If you are following along in your own development environment, rather than [Colab](https://colab.research.google.com/github/tensorflow/docs/blob/master/site/en/tutorials/quickstart/beginner.ipynb), see the [install guide](https://www.tensorflow.org/install) for setting up TensorFlow for development.

Note: Make sure you have upgraded to the latest `pip` to install the TensorFlow 2 package if you are using your own development environment. See the [install guide](https://www.tensorflow.org/install) for details.

## Load a dataset

Load and prepare the MNIST dataset. The pixel values of the images range from 0 through 255. Scale these values to a range of 0 to 1 by dividing the values by `255.0`. This also converts the sample data from integers to floating-point numbers:

In [4]:
mnist = tf.keras.datasets.mnist

(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0

11490434/11490434 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


## Build a machine learning model

Build a `tf.keras.Sequential` model:

In [5]:
model = tf.keras.models.Sequential([
  tf.keras.layers.Flatten(input_shape=(28, 28)),
  tf.keras.layers.Dense(128, activation='relu'),
  tf.keras.layers.Dropout(0.2),
  tf.keras.layers.Dense(10)
])

/usr/local/lib/python3.11/dist-packages/keras/src/layers/reshaping/flatten.py:37: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


[`Sequential`](https://www.tensorflow.org/guide/keras/sequential_model) is useful for stacking layers where each layer has one input [tensor](https://www.tensorflow.org/guide/tensor) and one output tensor. Layers are functions with a known mathematical structure that can be reused and have trainable variables. Most TensorFlow models are composed of layers. This model uses the [`Flatten`](https://www.tensorflow.org/api_docs/python/tf/keras/layers/Flatten), [`Dense`](https://www.tensorflow.org/api_docs/python/tf/keras/layers/Dense), and [`Dropout`](https://www.tensorflow.org/api_docs/python/tf/keras/layers/Dropout) layers.

For each example, the model returns a vector of [logits](https://developers.google.com/machine-learning/glossary#logits) or [log-odds](https://developers.google.com/machine-learning/glossary#log-odds) scores, one for each class.

In [6]:
predictions = model(x_train[:1]).numpy()
predictions

array([[ 0.43283644,  0.70794404, -0.28912976, -0.7065731 ,  0.08774929,
         0.16744813,  0.02478191,  0.36720178,  0.15605627,  0.3131232 ]],
      dtype=float32)

The `tf.nn.softmax` function converts these logits to *probabilities* for each class:

In [7]:
tf.nn.softmax(predictions).numpy()

array([[0.1274802 , 0.16784966, 0.06192939, 0.04079467, 0.09027621,
        0.0977656 , 0.08476703, 0.11938177, 0.09665819, 0.11309723]],
      dtype=float32)

Note: It is possible to bake the `tf.nn.softmax` function into the activation function for the last layer of the network. While this can make the model output more directly interpretable, this approach is discouraged as it's impossible to provide an exact and numerically stable loss calculation for all models when using a softmax output.

Define a loss function for training using `losses.SparseCategoricalCrossentropy`:

In [8]:
loss_fn = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)

The loss function takes a vector of ground truth values and a vector of logits and returns a scalar loss for each example. This loss is equal to the negative log probability of the true class: The loss is zero if the model is sure of the correct class.

This untrained model gives probabilities close to random (1/10 for each class), so the initial loss should be close to `-tf.math.log(1/10) ~= 2.3`.

In [9]:
loss_fn(y_train[:1], predictions).numpy()

np.float32(2.3251824)

Before you start training, configure and compile the model using Keras `Model.compile`. Set the [`optimizer`](https://www.tensorflow.org/api_docs/python/tf/keras/optimizers) class to `adam`, set the `loss` to the `loss_fn` function you defined earlier, and specify a metric to be evaluated for the model by setting the `metrics` parameter to `accuracy`.

In [10]:
model.compile(optimizer='adam',
              loss=loss_fn,
              metrics=['accuracy'])

## Train and evaluate your model

Use the `Model.fit` method to adjust your model parameters and minimize the loss:

In [11]:
model.fit(x_train, y_train, epochs=5)

Epoch 1/5
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 9s 4ms/step - accuracy: 0.8579 - loss: 0.4818
Epoch 2/5
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 8s 4ms/step - accuracy: 0.9546 - loss: 0.1520
Epoch 3/5
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 6s 3ms/step - accuracy: 0.9668 - loss: 0.1058
Epoch 4/5
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 10s 3ms/step - accuracy: 0.9747 - loss: 0.0857
Epoch 5/5
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 11s 4ms/step - accuracy: 0.9753 - loss: 0.0759


The `Model.evaluate` method checks the model's performance, usually on a [validation set](https://developers.google.com/machine-learning/glossary#validation-set) or [test set](https://developers.google.com/machine-learning/glossary#test-set).

In [12]:
model.evaluate(x_test,  y_test, verbose=2)

313/313 - 1s - 2ms/step - accuracy: 0.9778 - loss: 0.0733


[0.07333679497241974, 0.9778000116348267]

The image classifier is now trained to ~98% accuracy on this dataset. To learn more, read the [TensorFlow tutorials](https://www.tensorflow.org/tutorials/).

If you want your model to return a probability, you can wrap the trained model, and attach the softmax to it:

In [13]:
probability_model = tf.keras.Sequential([
  model,
  tf.keras.layers.Softmax()
])

In [14]:
probability_model(x_test[:5])

<tf.Tensor: shape=(5, 10), dtype=float32, numpy=
array([[1.3059147e-07, 9.1547712e-09, 6.0929724e-06, 3.0468131e-04,
        1.1037149e-10, 9.9673452e-09, 4.4042448e-12, 9.9968779e-01,
        4.8919827e-07, 8.1397201e-07],
       [1.5015758e-08, 1.3068494e-04, 9.9981326e-01, 5.5497363e-05,
        3.8651973e-15, 4.8644029e-07, 2.3479301e-09, 2.9867547e-14,
        1.2681181e-07, 7.9097512e-15],
       [2.4123437e-08, 9.9943465e-01, 2.3236984e-05, 2.4129768e-06,
        3.2162101e-05, 1.4014710e-06, 3.8875044e-07, 4.2132114e-04,
        8.2813458e-05, 1.4840549e-06],
       [9.9980277e-01, 1.6556066e-10, 4.4272542e-06, 1.6636875e-07,
        5.8593866e-07, 1.5290934e-06, 2.4464268e-06, 1.0544894e-06,
        1.2590958e-07, 1.8687986e-04],
       [3.1779671e-06, 8.5433715e-07, 2.7548851e-04, 4.3762293e-07,
        9.9340194e-01, 9.3690160e-06, 1.3354093e-05, 3.1453554e-04,
        2.6405207e-06, 5.9781866e-03]], dtype=float32)>

## Conclusion

Congratulations! You have trained a machine learning model using a prebuilt dataset using the [Keras](https://www.tensorflow.org/guide/keras/overview) API.

For more examples of using Keras, check out the [tutorials](https://www.tensorflow.org/tutorials/keras/). To learn more about building models with Keras, read the [guides](https://www.tensorflow.org/guide/keras). If you want learn more about loading and preparing data, see the tutorials on [image data loading](https://www.tensorflow.org/tutorials/load_data/images) or [CSV data loading](https://www.tensorflow.org/tutorials/load_data/csv).


# TEST 2

In [ ]:
import pandas as pd

df3 = pd.read_excel('TEST1.xlsx')
df4 = pd.read_excel('TEST2.xlsx')

print("First new file loaded:")
display(df3.head())

print("\nSecond new file loaded:")
display(df4.head())

In [ ]:
merged_df_new = df3.merge(df4, how='outer', indicator=True)

differences_new = merged_df_new[merged_df_new['_merge'] != 'both']

print("Differences found between the new files:")
display(differences_new)

# TEST 3

In [28]:
from google.colab import files

uploaded = files.upload()

for fn in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(
      name=fn, length=len(uploaded[fn])))

Saving Test3.xlsx to Test3 (3).xlsx
User uploaded file "Test3 (3).xlsx" with length 9493 bytes


In [29]:
import pandas as pd
import io

# Assuming the uploaded file is the first one in the uploaded dictionary
# and the email addresses are in a column named 'Email'
uploaded_file_name = list(uploaded.keys())[0]
df = pd.read_excel(io.BytesIO(uploaded[uploaded_file_name]))

# Extract email addresses, assuming they are in a column named 'Email'
if 'Email' in df.columns:
    email_list = df['Email'].tolist()
    print("Email addresses extracted:")
    print(email_list)
else:
    print("Error: 'Email' column not found in the Excel file.")
    email_list = []

Email addresses extracted:
['sanjanatelkar18@gmail.com', 'venkateshtelkar118@gmail.com']


In [30]:
email_subject = input("Enter the email subject: ")
email_body = input("Enter the email message: ")
send_time_str = input("Enter the desired sending time (YYYY-MM-DD HH:MM:SS): ")

# You might need to parse the send_time_str into a datetime object
# depending on how you plan to schedule the email sending.
print("\nEmail details collected:")
print(f"Subject: {email_subject}")
print(f"Body: {email_body}")
print(f"Send Time: {send_time_str}")

Enter the email subject: Birthday Party Invitation
Enter the email message: Hi everyone, please attend
Enter the desired sending time (YYYY-MM-DD HH:MM:SS): 2025-07-07 17:50:00

Email details collected:
Subject: Birthday Party Invitation
Body: Hi everyone, please attend
Send Time: 2025-07-07 17:50:00


**Security Warning:** Storing your email password directly in the code is not recommended. Consider using Colab's Secrets feature to store your password securely. You can access it by clicking the "🔑" icon in the left sidebar.

In [31]:
import smtplib
from email.mime.text import MIMEText
from email.mime.multipart import MIMEMultipart
import time

email_address = "abc@gmail.com"
email_password = "dhjksj@123"

smtp_server = "smtp.gmail.com"
smtp_port = 587 # For TLS

if email_list:
    print(f"Attempting to send email to {len(email_list)} recipients...")
    try:
        server = smtplib.SMTP(smtp_server, smtp_port)
        server.starttls() # Secure the connection
        server.login(email_address, email_password)

        for recipient_email in email_list:
            msg = MIMEMultipart()
            msg['From'] = email_address
            msg['To'] = recipient_email
            msg['Subject'] = email_subject

            msg.attach(MIMEText(email_body, 'plain'))

            text = msg.as_string()
            server.sendmail(email_address, recipient_email, text)
            print(f"Email sent to {recipient_email}")
            time.sleep(1) # Add a small delay between sending emails

        server.quit()
        print("\nAll emails sent successfully!")

    except Exception as e:
        print(f"An error occurred: {e}")
else:
    print("No email addresses found to send emails.")

Attempting to send email to 2 recipients...
An error occurred: (535, b'5.7.8 Username and Password not accepted. For more information, go to\n5.7.8  https://support.google.com/mail/?p=BadCredentials 8926c6da1cb9f-503b5bdf998sm1641854173.83 - gsmtp')
